# London

## 1. import libraries

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import community
import osmnx as ox

## 2. load shapefile

In [ ]:
Lon_shp = 'GLA/shp/GLA_Small_p.shp'
G=nx.read_shp(Lon_shp,simplify=True) 

In [ ]:
#nx.write_shp(G, 'GLA/shp/nx/london.shp')

In [ ]:
for (i,j,w) in G.edges(data=True):
    del G.edges()[i,j]['Json']
    del G.edges()[i,j]['ShpName']
    del G.edges()[i,j]['Wkb']
    del G.edges()[i,j]['Wkt']
nx.write_edgelist(G,'GLA/test.edgelist',delimiter=',',data=True)

In [2]:
# step here needed to make edgelist into pandas dataframe
df=pd.read_csv('GLA/test.edgelist',names=['source_lon','source_lat','target_lon','target_lat','ID','VD','sky','bldg','road','tree','Cl','NetCl','FP','CH'])

In [3]:
df.head()

,source_lon,source_lat,target_lon,target_lat,ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH
0,(530462.0,200000.03),(530498.99,200211.97),{'GLA_ID': 1.0,'res_pred': None,'Sky': None,'Building': None,'Road': None,'Tree': None,'D_LON_Cl': None,'D_LON_NETC': None,'F_act_prob': None,'Ax_Choice': None}
1,(530498.99,200211.97),(530679.97,200093.02),{'GLA_ID': 4.0,'res_pred': 0.03550648,'Sky': 29889.0,'Building': 37920.0,'Road': 3844.0,'Tree': 43543.0,'D_LON_Cl': 0.0,'D_LON_NETC': 1.0,'F_act_prob': 0.000282750464976,'Ax_Choice': 57219784.0}
2,(531525.98,200009.05),(531528.98,200000.0),{'GLA_ID': 2.0,'res_pred': -0.08743627,'Sky': 56167.0,'Building': 23552.0,'Road': 30815.0,'Tree': 15731.0,'D_LON_Cl': 2.0,'D_LON_NETC': 1.0,'F_act_prob': 0.001773398253135,'Ax_Choice': 42399540.0}
3,(531528.98,200000.0),(531580.97,199817.97),{'GLA_ID': 65378.0,'res_pred': -0.037550144,'Sky': 48892.0,'Building': 25853.0,'Road': 38137.0,'Tree': 22750.0,'D_LON_Cl': 2.0,'D_LON_NETC': 1.0,'F_act_prob': 0.000493181345519,'Ax_Choice': 42399540.0}
4,(530191.96,200566.04),(530498.99,200211.97),{'GLA_ID': 3.0,'res_pred': 0.007293992,'Sky': 30940.0,'Building': 4670.0,'Road': 24543.0,'Tree': 49084.0,'D_LON_Cl': 0.0,'D_LON_NETC': 1.0,'F_act_prob': 1.0504512829e-05,'Ax_Choice': 81773808.0}


In [4]:
#df[(df['ID'].isnull()==True)]
#df=df.fillna(0, inplace=True)
#nulls = df.isnull().sum()
#nulls[nulls > 0]

In [5]:
df['source_lon'] = df['source_lon'].str.replace('(','')
df['target_lon'] = df['target_lon'].str.replace('(','')
df['source_lat'] = df['source_lat'].str.replace(')','')
df['target_lat'] = df['target_lat'].str.replace(')','')
df['ID'] = df['ID'].str.replace("{'GLA_ID':",'')
df['VD'] = df['VD'].str.replace("'res_pred':",'')
df['sky'] = df['sky'].str.replace("'Sky':",'')
df['bldg'] = df['bldg'].str.replace("'Building':",'')
df['road'] = df['road'].str.replace("'Road':",'')
df['tree'] = df['tree'].str.replace("'Tree':",'')
df['Cl'] = df['Cl'].str.replace("'D_LON_Cl': ",'')
df['NetCl'] = df['NetCl'].str.replace("'D_LON_NETC': ",'')
df['FP'] = df['FP'].str.replace("'F_act_prob':",'')
df['CH'] = df['CH'].str.replace("'Ax_Choice':",'')
df['CH'] = df['CH'].str.replace("}",'')
df['ID'] = df['ID'].str.replace("'",'')

In [6]:
df['VD']=df['VD'].str.replace('None','0')
df['sky']=df['sky'].str.replace('None','0')
df['bldg']=df['bldg'].str.replace('None','0')
df['road']=df['road'].str.replace('None','0')
df['tree']=df['tree'].str.replace('None','0')
df['Cl']=df['Cl'].str.replace('None','0')
df['NetCl']=df['NetCl'].str.replace('None','0')
df['FP']=df['FP'].str.replace('None','0')
df['CH']=df['CH'].str.replace('None','0')


In [7]:
df['source_lon']=df['source_lon'].astype(float)
df['source_lat']=df['source_lat'].astype(float)
df['target_lon']=df['target_lon'].astype(float)
df['target_lat']=df['target_lat'].astype(float)
df['ID']=df['ID'].astype(float)
df['VD']=df['VD'].astype(float)
df['sky']=df['sky'].astype(float)
df['bldg']=df['bldg'].astype(float)
df['road']=df['road'].astype(float)
df['tree']=df['tree'].astype(float)
df['Cl']=df['Cl'].astype(float)
df['NetCl']=df['NetCl'].astype(float)
df['FP']=df['FP'].astype(float)
df['CH']=df['CH'].astype(float)

In [8]:
df['CH']=np.log(df['CH']+2)

### create nodelist

In [9]:
# creates nodelist
df2=pd.read_csv('GLA/nodelist.csv')
df2.head()
df3=df2.drop_duplicates()
df4=df3.reset_index()
del df4['index']
df5=df4.reset_index()
df5 = df5.rename(columns={'index': 'ID'})
df5 = df5.rename(columns={'X': 'lon','Y': 'lat'})

In [10]:
precision = 3
df['source_lon']=df['source_lon'].round(decimals = precision)
df['source_lat']=df['source_lat'].round(decimals = precision)
df['target_lon']=df['target_lon'].round(decimals = precision)
df['target_lat']=df['target_lat'].round(decimals = precision)
df5['lon'] = df5['lon'].round(decimals = precision)
df5['lat'] = df5['lat'].round(decimals = precision)


In [11]:
# creates edgelist
df6 = pd.merge(df,df5, how='left',left_on=['source_lon','source_lat'], right_on =['lon','lat'])
del df6['lon']
del df6['lat']
df7 = df6.rename(columns={'ID_x': 'Edge_ID','ID_y': 'source_ID'})
df8 = pd.merge(df7,df5, how='left',left_on=['target_lon','target_lat'], right_on =['lon','lat'])
df8 = df8.rename(columns={'ID': 'target_ID'})
del df8['lon']
del df8['lat']

#del df8['ID_x']
#del df8['ID_y']

## 3. create dual graph

In [12]:
df_nodelist=df5

In [13]:
df5.head()

,ID,lon,lat
0,0,530462.00,200000.03
1,1,530498.99,200211.97
2,2,531525.98,200009.05
3,3,531528.98,200000.00
4,4,530191.96,200566.04


In [14]:
df_edgelist=df8
del df_edgelist['source_lon']
del df_edgelist['source_lat']
del df_edgelist['target_lon']
del df_edgelist['target_lat']

In [15]:
df_edgelist.head()

,Edge_ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH,source_ID,target_ID
0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.693147,0,1
1,4.0,0.035506,29889.0,37920.0,3844.0,43543.0,0.0,1.0,0.000283,17.862410,1,5
2,2.0,-0.087436,56167.0,23552.0,30815.0,15731.0,2.0,1.0,0.001773,17.562648,2,3
3,65378.0,-0.037550,48892.0,25853.0,38137.0,22750.0,2.0,1.0,0.000493,17.562648,3,56957
4,3.0,0.007294,30940.0,4670.0,24543.0,49084.0,0.0,1.0,0.000011,18.219468,4,1


In [112]:
G=nx.from_pandas_dataframe(df_edgelist, 'source_ID', 'target_ID')
DG=nx.line_graph(G)
nx.write_edgelist(DG,'GLA/dual_graph.edgelist',delimiter=',',data=True)


In [16]:
df_dualgraph=pd.read_csv('GLA/dual_graph.edgelist',index_col=False,names=['source1','source2','target1','target2'])
df_dualgraph.head()

,source1,source2,target1,target2
0,(1.0,4.0),(0.0,1.0)
1,(1.0,4.0),(1.0,5.0)
2,(21.0,22.0),(22.0,4023.0)
3,(26.0,4024.0),(25.0,26.0)
4,(33.0,34.0),(34.0,4028.0)


In [17]:
df_dualgraph['source1'] = df_dualgraph['source1'].str.replace('(','')
df_dualgraph['target1'] = df_dualgraph['target1'].str.replace('(','')
df_dualgraph['source2'] = df_dualgraph['source2'].str.replace(')','')
df_dualgraph['target2'] = df_dualgraph['target2'].str.replace(')','')

df_dualgraph['source1'] = df_dualgraph['source1'].astype(float)
df_dualgraph['target1'] = df_dualgraph['target1'].astype(float)
df_dualgraph['source2'] = df_dualgraph['source2'].astype(float)
df_dualgraph['target2'] = df_dualgraph['target2'].astype(float)


In [18]:
# for saving and loading
#df_nodelist.to_csv('networkx/nodelist.csv')
#df_edgelist.to_csv('GLA/edgelist.csv',index=False)
df_edgelist = pd.read_csv('GLA/edgelist.csv')
#del df_edgelist['Unnamed: 0']

In [19]:
df_dualgraph.head()

,source1,source2,target1,target2
0,1.0,4.0,0.0,1.0
1,1.0,4.0,1.0,5.0
2,21.0,22.0,22.0,4023.0
3,26.0,4024.0,25.0,26.0
4,33.0,34.0,34.0,4028.0


In [20]:
df_edgelist

,Edge_ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH,source_ID,target_ID
0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.693147,0,1
1,4.0,0.035506,29889.0,37920.0,3844.0,43543.0,0.0,1.0,0.000283,17.862410,1,5
2,2.0,-0.087436,56167.0,23552.0,30815.0,15731.0,2.0,1.0,0.001773,17.562648,2,3
3,65378.0,-0.037550,48892.0,25853.0,38137.0,22750.0,2.0,1.0,0.000493,17.562648,3,56957
4,3.0,0.007294,30940.0,4670.0,24543.0,49084.0,0.0,1.0,0.000011,18.219468,4,1
5,5.0,0.059349,37829.0,16689.0,25528.0,43527.0,1.0,1.0,0.000045,13.231695,5,6
6,6.0,-0.207564,59595.0,15820.0,39686.0,8680.0,4.0,1.0,0.000153,17.793498,6,2
7,7.0,-0.037117,64240.0,24653.0,45085.0,5702.0,2.0,1.0,0.009025,14.259435,7,8
8,4402.0,-0.049668,72208.0,34317.0,25350.0,35.0,2.0,1.0,0.000032,14.259435,8,3966
9,8.0,-0.171608,50002.0,59769.0,28931.0,1725.0,2.0,3.0,0.713976,17.243651,9,10


## 4. make directional edgelist from dataframe

In [21]:
df_edgelist1=pd.read_csv('GLA/edgelist.csv')

In [22]:
df_edgelist2=pd.read_csv('GLA/edgelist.csv')

In [23]:
#del df_edgelist1['source_lon']
#del df_edgelist1['source_lat']
#del df_edgelist1['target_lon']
#del df_edgelist1['target_lat']
#del df_edgelist2['source_lon']
#del df_edgelist2['source_lat']
#del df_edgelist2['target_lon']
#del df_edgelist2['target_lat']
#del df_edgelist1['Unnamed: 0']
#del df_edgelist2['Unnamed: 0']

In [24]:
df_edgelist1.head()

,Edge_ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH,source_ID,target_ID
0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.693147,0,1
1,4.0,0.035506,29889.0,37920.0,3844.0,43543.0,0.0,1.0,0.000283,17.862410,1,5
2,2.0,-0.087436,56167.0,23552.0,30815.0,15731.0,2.0,1.0,0.001773,17.562648,2,3
3,65378.0,-0.037550,48892.0,25853.0,38137.0,22750.0,2.0,1.0,0.000493,17.562648,3,56957
4,3.0,0.007294,30940.0,4670.0,24543.0,49084.0,0.0,1.0,0.000011,18.219468,4,1


In [25]:
df_edgelist2.head()

,Edge_ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH,source_ID,target_ID
0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.693147,0,1
1,4.0,0.035506,29889.0,37920.0,3844.0,43543.0,0.0,1.0,0.000283,17.862410,1,5
2,2.0,-0.087436,56167.0,23552.0,30815.0,15731.0,2.0,1.0,0.001773,17.562648,2,3
3,65378.0,-0.037550,48892.0,25853.0,38137.0,22750.0,2.0,1.0,0.000493,17.562648,3,56957
4,3.0,0.007294,30940.0,4670.0,24543.0,49084.0,0.0,1.0,0.000011,18.219468,4,1


In [26]:
df_edgelist.head()

,Edge_ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH,source_ID,target_ID
0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.693147,0,1
1,4.0,0.035506,29889.0,37920.0,3844.0,43543.0,0.0,1.0,0.000283,17.862410,1,5
2,2.0,-0.087436,56167.0,23552.0,30815.0,15731.0,2.0,1.0,0.001773,17.562648,2,3
3,65378.0,-0.037550,48892.0,25853.0,38137.0,22750.0,2.0,1.0,0.000493,17.562648,3,56957
4,3.0,0.007294,30940.0,4670.0,24543.0,49084.0,0.0,1.0,0.000011,18.219468,4,1


In [27]:
df_edgelist2=df_edgelist2.rename(columns={'source_ID':'target_ID2','target_ID': 'source_ID2'})
df_edgelist2=df_edgelist2.rename(columns={'target_ID2':'target_ID','source_ID2': 'source_ID'})
#df_edgelist2 = df_edgelist2[['Edge_ID', 'FP','source_ID', 'target_ID']]

In [28]:
df_both_edgelist=pd.concat([df_edgelist,df_edgelist2])
#df_both_edgelist.to_csv('GLA/Both_edgelist.csv',index=False)

In [29]:
df_both_edgelist.head()

,CH,Cl,Edge_ID,FP,NetCl,VD,bldg,road,sky,source_ID,target_ID,tree
0,0.693147,0.0,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0,1,0.0
1,17.862410,0.0,4.0,0.000283,1.0,0.035506,37920.0,3844.0,29889.0,1,5,43543.0
2,17.562648,2.0,2.0,0.001773,1.0,-0.087436,23552.0,30815.0,56167.0,2,3,15731.0
3,17.562648,2.0,65378.0,0.000493,1.0,-0.037550,25853.0,38137.0,48892.0,3,56957,22750.0
4,18.219468,0.0,3.0,0.000011,1.0,0.007294,4670.0,24543.0,30940.0,4,1,49084.0


## 5. key merge between dual graph and edgelist

In [30]:
#df_both_edgelist = pd.read_csv('GLA/Both_edgelist.csv')

In [31]:
df_both_edgelist.head()

,CH,Cl,Edge_ID,FP,NetCl,VD,bldg,road,sky,source_ID,target_ID,tree
0,0.693147,0.0,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0,1,0.0
1,17.862410,0.0,4.0,0.000283,1.0,0.035506,37920.0,3844.0,29889.0,1,5,43543.0
2,17.562648,2.0,2.0,0.001773,1.0,-0.087436,23552.0,30815.0,56167.0,2,3,15731.0
3,17.562648,2.0,65378.0,0.000493,1.0,-0.037550,25853.0,38137.0,48892.0,3,56957,22750.0
4,18.219468,0.0,3.0,0.000011,1.0,0.007294,4670.0,24543.0,30940.0,4,1,49084.0


In [32]:
#df.head()

In [33]:
precision = 3
df['source_lon']=df['source_lon'].round(decimals = precision)
df['source_lat']=df['source_lat'].round(decimals = precision)
df['target_lon']=df['target_lon'].round(decimals = precision)
df['target_lat']=df['target_lat'].round(decimals = precision)
df5['lon'] = df5['lon'].round(decimals = precision)
df5['lat'] = df5['lat'].round(decimals = precision)

In [34]:
df_merge=pd.merge(df_dualgraph,df_both_edgelist,how='left',left_on=['source1','source2'], right_on =['source_ID','target_ID'])
del df_merge['source_ID']
del df_merge['target_ID']


In [35]:
df_merge = df_merge.rename(columns={'Cl':'source_Cl','NetCl': 'source_netCl','Edge_ID': 'source_Edge_ID','FP':'source_FP','sky':'source_sky','bldg':'source_bldg','road':'source_road','tree':'source_tree','VD':'source_VD','CH':'source_CH'})

In [36]:
df_merge=pd.merge(df_merge,df_both_edgelist,how='left',left_on=['target1','target2'], right_on =['source_ID','target_ID'])
del df_merge['source_ID']
del df_merge['target_ID']


In [37]:
df_merge = df_merge.rename(columns={'Cl':'target_Cl','NetCl': 'target_netCl','Edge_ID': 'target_Edge_ID','FP':'target_FP','sky':'target_sky','bldg':'target_bldg','road':'target_road','tree':'target_tree','VD':'target_VD','CH':'target_CH'})

In [38]:
df_merge['netCl'] = np.abs(df_merge['source_netCl']-df_merge['target_netCl'])
df_merge['netCl']= 1/(df_merge['netCl']+1)
df_merge['Cl'] = np.abs(df_merge['source_Cl']-df_merge['target_Cl'])
df_merge['Cl']=1/(df_merge['Cl']+1)

df_merge['sky'] = np.abs(df_merge['source_sky']-df_merge['target_sky'])
df_merge['sky']= 1/(df_merge['sky']+1)
df_merge['bldg'] = np.abs(df_merge['source_bldg']-df_merge['target_bldg'])
df_merge['bldg']=1/(df_merge['bldg']+1)
df_merge['tree'] = np.abs(df_merge['source_tree']-df_merge['target_tree'])
df_merge['tree']= 1/(df_merge['tree']+1)
df_merge['road'] = np.abs(df_merge['source_road']-df_merge['target_road'])
df_merge['road']=1/(df_merge['road']+1)
df_merge['sum_road'] = np.abs(df_merge['source_road']+df_merge['target_road'])
df_merge['sum_road']=1/(df_merge['road']+1)
df_merge['FP'] = np.abs(df_merge['source_FP']-df_merge['target_FP'])
df_merge['FP']= 1/(df_merge['FP']+1)
df_merge['VD'] = np.abs(df_merge['source_VD']-df_merge['target_VD'])
df_merge['VD']= 1/(df_merge['VD']+1)

df_merge['CH'] = np.abs(df_merge['source_CH']+df_merge['target_CH'])
df_merge['CH']= 1/(df_merge['CH']+1)
#df_merge['CH'] = (df_merge['source_CH']+df_merge['target_CH'])
#df_merge['CH']= (1/(df_merge['CH']+1))*100

#df_BC = (df_BC - df_BC.min()) / (df_BC.max() - df_BC.min())
#df_BC=df_BC.clip(lower=0.01, upper=0.99)




In [39]:
#df_merge.to_csv('GLA/df_merge.csv',index=False)
#df_merge=pd.read_csv('London/df_merge.csv')
#del df_merge['Unnamed: 0']

In [40]:
del df_merge['source1']
del df_merge['source2']
del df_merge['target1']
del df_merge['target2']
del df_merge['source_Cl']
del df_merge['source_netCl']
del df_merge['target_Cl']
del df_merge['target_netCl']
del df_merge['source_road']
del df_merge['source_sky']
del df_merge['source_bldg']
del df_merge['source_tree']
del df_merge['target_road']
del df_merge['target_sky']
del df_merge['target_bldg']
del df_merge['target_tree']
del df_merge['source_VD']
del df_merge['source_FP']
del df_merge['target_VD']
del df_merge['target_FP']

del df_merge['source_CH']
del df_merge['target_CH']




In [157]:
df_merge.to_csv('GLA/GLA_edgelist.csv',index=False)

In [41]:
df_merge=pd.read_csv('GLA/GLA_edgelist.csv')

In [41]:
df_merge.head()

,source_Edge_ID,target_Edge_ID,netCl,Cl,sky,bldg,tree,road,sum_road,FP,VD,CH
0,3.0,1.0,0.5,1.000000,0.000032,0.000214,0.000020,0.000041,0.999959,0.999989,0.992759,0.050219
1,3.0,4.0,1.0,1.000000,0.000951,0.000030,0.000180,0.000048,0.999952,0.999728,0.972562,0.026967
2,14.0,4451.0,1.0,0.500000,0.000082,0.000204,0.000063,0.000037,0.999963,0.994478,0.962999,0.035517
3,4453.0,16.0,0.5,0.333333,0.000584,0.000151,0.000153,0.000213,0.999787,0.987454,0.922827,0.419060
4,20.0,4457.0,1.0,0.333333,0.000054,0.000117,0.003257,0.000098,0.999902,0.999828,0.947112,0.419060


### optional: calculate betweenness_centrality

In [163]:
#DG=nx.from_pandas_dataframe(df_merge, 'source_Edge_ID', 'target_Edge_ID', 'FP')
#bc=nx.betweenness_centrality(DG)
# needs to merge it back to dataframe
#df_BC=pd.DataFrame(list(bc.items()), columns=['nodes', 'BC'])
#df_BC = df_BC.set_index('nodes')
#df_BC = (df_BC - df_BC.min()) / (df_BC.max() - df_BC.min())
#df_BC=df_BC.clip(lower=0.01, upper=0.99)
#df_all=pd.merge(left=df_merge,right=df_BC,left_on='source_Edge_ID',right_index=True)
#df_all=df_all.rename(columns={'BC':'source_BC'})
#df_all=pd.merge(left=df_all,right=df_BC,left_on='target_Edge_ID',right_index=True)
#df_all=df_all.rename(columns={'BC':'target_BC'})
#df_all['BC'] = np.abs(df_all['source_BC']+df_all['target_BC'])
#df_all['BC']= 1/(df_all['BC']+0.01)
#del df_all['source_BC']
#del df_all['target_BC']
#df_all.loc[(df_all['BC']<100)]

## 7. Dual graph communities using average choice of nodes

### dual edge weights = f(sum of source choice and target choice)

* if high choice on both streets = disconnected

* if low choice on both streets = connected

* this takes the choice from two edges 

* create dual graph based on source and target choice

* calculate modularity communities

In [390]:
df_merge.to_csv('GLA/final/df_merge.csv',index=False)
#df_merge=pd.read_csv('GLA/final/df_merge.csv')

In [161]:
DG=nx.from_pandas_dataframe(df_merge, 'source_Edge_ID', 'target_Edge_ID', 'CH')
partition = community.best_partition(DG)
partition_CH = community.best_partition(DG,weight='CH')
df_partition = pd.DataFrame(list(partition.items()), columns=['EdgeID', 'Com_Topo'])
df_partition_CH = pd.DataFrame(list(partition_CH.items()), columns=['EdgeID', 'Com_CH'])
df_com = pd.merge(df_partition,df_partition_CH)
df_com.to_csv('GLA/GLA_CH_communities.csv',index=False)

In [43]:
DG=nx.from_pandas_dataframe(df_merge, 'source_Edge_ID', 'target_Edge_ID', 'sum_road')
partition = community.best_partition(DG)
partition_CH = community.best_partition(DG,weight='sum_road')
df_partition = pd.DataFrame(list(partition.items()), columns=['EdgeID', 'Com_Topo'])
df_partition_CH = pd.DataFrame(list(partition_CH.items()), columns=['EdgeID', 'Com_Road'])
df_com = pd.merge(df_partition,df_partition_CH)
df_com.to_csv('GLA/GLA_Road_communities.csv',index=False)

In [45]:
DG=nx.from_pandas_dataframe(df_merge, 'source_Edge_ID', 'target_Edge_ID', 'FP')
partition = community.best_partition(DG)
partition_FP = community.best_partition(DG,weight='FP')
df_partition = pd.DataFrame(list(partition.items()), columns=['EdgeID', 'Com_Topo'])
df_partition_FP = pd.DataFrame(list(partition_FP.items()), columns=['EdgeID', 'Com_FP'])
df_com = pd.merge(df_partition,df_partition_FP)
df_com.to_csv('GLA/GLA_FP_communities.csv',index=False)

In [166]:
DG=nx.from_pandas_dataframe(df_merge, 'source_Edge_ID', 'target_Edge_ID', 'VD')
partition = community.best_partition(DG)
partition_VD = community.best_partition(DG,weight='VD')
df_partition = pd.DataFrame(list(partition.items()), columns=['EdgeID', 'Com_Topo'])
df_partition_VD = pd.DataFrame(list(partition_VD.items()), columns=['EdgeID', 'Com_VD'])
df_com = pd.merge(df_partition,df_partition_VD)
df_com.to_csv('GLA/GLA_VD_communities.csv',index=False)

In [382]:
# 42606 is approximately 20%
len(df_merge.loc[df_merge['bldg'] < 0.000048])

51674

In [383]:
df_merge['new_bldg']=np.NaN
df_merge.loc[df_merge['bldg'] < 0.000048, 'new_bldg'] = 1
df_merge.loc[df_merge['bldg'] >= 0.000048, 'new_bldg'] = 10

In [384]:
DG=nx.from_pandas_dataframe(df_merge, 'source_Edge_ID', 'target_Edge_ID', 'bldg')
partition = community.best_partition(DG)
partition_bldg = community.best_partition(DG,weight='bldg')
partition_new_bldg = community.best_partition(DG,weight='new_bldg')

df_partition = pd.DataFrame(list(partition.items()), columns=['EdgeID', 'Com_Topo'])
df_partition_bldg = pd.DataFrame(list(partition_bldg.items()), columns=['EdgeID', 'Com_bldg'])
df_partition_new_bldg = pd.DataFrame(list(partition_new_bldg.items()), columns=['EdgeID', 'Com_new_bldg'])

df_com = pd.merge(df_partition,df_partition_bldg)
df_com = pd.merge(df_com,df_partition_new_bldg)
df_com.to_csv('GLA/GLA_bldg_communities2.csv',index=False)

## 8. Primal graph communities using Street Choice

### primal edge weights = inverse depthmap choice

* use edge betweenness calculation as weights

* calculate modularity communities based on betweenness

In [170]:
df_edgelist['inv_CH']=1/(df_edgelist['CH'])
df_edgelist.head()

,Edge_ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH,source_ID,target_ID,inv_CH
0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.693147,0,1,1.442695
1,4.0,0.035506,29889.0,37920.0,3844.0,43543.0,0.0,1.0,0.000283,17.862410,1,5,0.055983
2,2.0,-0.087436,56167.0,23552.0,30815.0,15731.0,2.0,1.0,0.001773,17.562648,2,3,0.056939
3,65378.0,-0.037550,48892.0,25853.0,38137.0,22750.0,2.0,1.0,0.000493,17.562648,3,56957,0.056939
4,3.0,0.007294,30940.0,4670.0,24543.0,49084.0,0.0,1.0,0.000011,18.219468,4,1,0.054886


In [168]:
G=nx.from_pandas_dataframe(df_edgelist, 'source_ID', 'target_ID', 'inv_CH')
partition = community.best_partition(G)
partition_CH = community.best_partition(G,weight='inv_CH')
df_partition = pd.DataFrame(list(partition.items()), columns=['NodeID', 'Com_Topo'])
df_partition_CH = pd.DataFrame(list(partition_CH.items()), columns=['NodeID', 'Com_CH'])
df_com = pd.merge(df_partition,df_partition_CH)
df_com.to_csv('GLA/GLA_Node_CH_communities.csv',index=False)

## 9. Primal Graph Communities using avg street Choice of connected edges

* Averaging the edge weights onto nodes

* taking the average of betweenness from surrounding edges back onto the nodes


In [443]:
df_src_Ch=pd.DataFrame(df_edgelist.groupby(['source_ID'])['CH'].sum())
df_src_ct = pd.DataFrame(df_edgelist.groupby(['source_ID'])['CH'].count())
df_tgt_Ch=pd.DataFrame(df_edgelist.groupby(['target_ID'])['CH'].sum())
df_tgt_ct = pd.DataFrame(df_edgelist.groupby(['target_ID'])['CH'].count())
df_src = pd.merge(left=df_src_Ch, right=df_src_ct, how='left', left_index=True, right_index=True)
df_tgt = pd.merge(left=df_tgt_Ch, right=df_tgt_ct, how='left', left_index=True, right_index=True)

In [444]:
df_src = df_src.rename(columns={'CH_x':'CH','CH_y': 'Ct'})
df_tgt = df_tgt.rename(columns={'CH_x':'CH','CH_y': 'Ct'})
df_src = df_src.rename(columns={'CH':'src_CH', 'Ct': 'src_Ct'})
df_tgt = df_tgt.rename(columns={'CH':'tgt_CH', 'Ct': 'tgt_Ct'})
df_src=df_src.reset_index()
df_tgt=df_tgt.reset_index()

In [445]:
df_nodelist_density = pd.merge(left=df_nodelist, right=df_src, how='left', left_on='ID', right_on='source_ID')
df_nodelist_density = pd.merge(left=df_nodelist_density, right=df_tgt, how='left', left_on='ID', right_on='target_ID')
df_nodelist_density = df_nodelist_density.fillna(0)
df_nodelist_density['avg_CH']=(df_nodelist_density['src_CH']+df_nodelist_density['tgt_CH'])/(df_nodelist_density['src_Ct']+df_nodelist_density['tgt_Ct'])

In [446]:
del df_nodelist_density['source_ID']
del df_nodelist_density['target_ID']
del df_nodelist_density['src_CH']
del df_nodelist_density['tgt_CH']
del df_nodelist_density['src_Ct']
del df_nodelist_density['tgt_Ct']
df_lon_nodes=df_nodelist_density

* primal edge weights = mean(node_CH)

In [447]:
# need step here - average the weights from edges into nodes
del df_lon_nodes['lon']
del df_lon_nodes['lat']
df_edgelist2 = pd.merge(df_edgelist,df_lon_nodes,how='left',left_on='source_ID',right_on='ID')
df_edgelist2 = df_edgelist2.rename(columns={'avg_CH':'src_avg_CH'})

In [448]:
del df_edgelist2['ID']
del df_edgelist2['CH']
df_edgelist2 = pd.merge(df_edgelist2,df_lon_nodes,how='left',left_on='target_ID',right_on='ID')
df_edgelist2 = df_edgelist2.rename(columns={'avg_CH':'tgt_avg_CH'})
del df_edgelist2['ID']

In [449]:
# this takes the Difference between source choice in node 1 and target choice in node 2
df_edgelist2['Sim_CH']=1/(np.abs(df_edgelist2['src_avg_CH']-df_edgelist2['tgt_avg_CH'])+1)
df_edgelist2['Sum_CH']=1/(np.abs(df_edgelist2['src_avg_CH']+df_edgelist2['tgt_avg_CH'])+1)


In [450]:
DG=nx.from_pandas_dataframe(df_edgelist2, 'source_ID', 'target_ID')
partition = community.best_partition(DG)
DG=nx.from_pandas_dataframe(df_edgelist2, 'source_ID', 'target_ID', 'Sim_CH')
partition_CH = community.best_partition(DG,weight='Sim_CH')
DG=nx.from_pandas_dataframe(df_edgelist2, 'source_ID', 'target_ID', 'Sum_CH')
partition_CHS = community.best_partition(DG,weight='Sum_CH')
df_partition = pd.DataFrame(list(partition.items()), columns=['NodeID', 'Com_Topo'])
df_partition_CH = pd.DataFrame(list(partition_CH.items()), columns=['NodeID', 'Com_CH'])
df_partition_CHS = pd.DataFrame(list(partition_CHS.items()), columns=['NodeID', 'Com_CHS'])
df_com = pd.merge(df_partition,df_partition_CH)
df_com = pd.merge(df_com,df_partition_CHS)

In [451]:
df_com.to_csv('GLA/London_primal_node_communities2.csv',index=False)

In [462]:
df_edgelist3=df_edgelist2.sample(frac=1)
df_edgelist3=df_edgelist3.reset_index()

In [463]:
DG=nx.from_pandas_dataframe(df_edgelist3, 'source_ID', 'target_ID')
partition = community.best_partition(DG)
DG=nx.from_pandas_dataframe(df_edgelist3, 'source_ID', 'target_ID', 'Sim_CH')
partition_CH = community.best_partition(DG,weight='Sim_CH')
DG=nx.from_pandas_dataframe(df_edgelist3, 'source_ID', 'target_ID', 'Sum_CH')
partition_CHS = community.best_partition(DG,weight='Sum_CH')
df_partition = pd.DataFrame(list(partition.items()), columns=['NodeID', 'Com_Topo'])
df_partition_CH = pd.DataFrame(list(partition_CH.items()), columns=['NodeID', 'Com_CH'])
df_partition_CHS = pd.DataFrame(list(partition_CHS.items()), columns=['NodeID', 'Com_CHS'])
df_com = pd.merge(df_partition,df_partition_CH)
df_com = pd.merge(df_com,df_partition_CHS)
df_com.to_csv('GLA/London_primal_node_communities4.csv',index=False)

## 10. Dual Graph communities using average junction node choice from connected edges

* take node weights of average edge choice as dual graph weights to calculate modularity

* using junction node weights in dual graph edges 

In [72]:
new_df=df_edgelist2[['Edge_ID','source_ID','target_ID']]
new_df2=df_merge[['source_Edge_ID','target_Edge_ID']]
new_df3=pd.merge(new_df2,new_df,how='left',left_on='source_Edge_ID',right_on='Edge_ID')
new_df3=new_df3.rename(columns={'source_ID':'src_Edge_s1','target_ID':'src_Edge_t1'})
del new_df3['Edge_ID']
new_df4=pd.merge(new_df3,new_df,how='left',left_on='target_Edge_ID',right_on='Edge_ID')
new_df4=new_df4.rename(columns={'source_ID':'tgt_Edge_s2','target_ID':'tgt_Edge_t2'})
del new_df4['Edge_ID']
new_df4['freq_ID']=np.NaN

In [ ]:
# this gets the most frequent node which is the junction of the two edge
for i in new_df4.index:
    value=new_df4[['src_Edge_s1','src_Edge_t1','tgt_Edge_s2','tgt_Edge_t2']].loc[i].mode()
    new_df4.loc[(i,'freq_ID')]=value[0]

# delete the source edge source/target node and target edge source/target note
del new_df4['src_Edge_s1']
del new_df4['src_Edge_t1']
del new_df4['tgt_Edge_s2']
del new_df4['tgt_Edge_t2']




In [ ]:
# now get the choice value of the junction node as the choice value 
new_df4=pd.merge(new_df4,df_lon_nodes,how='left',left_on='freq_ID',right_on='ID')

In [341]:
new_df4['inv_CH']=1/new_df4['avg_CH']

* Dual Graph Communities using average node choice

In [343]:
# now calculate modularity optimisation based on the junction nodes
DG=nx.from_pandas_dataframe(new_df4, 'source_Edge_ID', 'target_Edge_ID')
partition = community.best_partition(DG)
DG=nx.from_pandas_dataframe(new_df4, 'source_Edge_ID', 'target_Edge_ID', 'avg_CH')
partition_CH = community.best_partition(DG,weight='avg_CH')
DG=nx.from_pandas_dataframe(new_df4, 'source_Edge_ID', 'target_Edge_ID', 'inv_CH')
partition_inv_CH = community.best_partition(DG,weight='inv_CH')
df_partition = pd.DataFrame(list(partition.items()), columns=['NodeID', 'Com_Topo'])
df_partition_CH = pd.DataFrame(list(partition_CH.items()), columns=['NodeID', 'Com_CH'])
df_partition_inv_CH = pd.DataFrame(list(partition_inv_CH.items()), columns=['NodeID', 'Com_invCH'])
df_com = pd.merge(df_partition,df_partition_CH)
df_com = pd.merge(df_com,df_partition_inv_CH)

In [345]:
df_com.to_csv('GLA/GLA_dual_communities_nodes_CH.csv')

## 11. Dual Graph communities using junction node max choice

In [145]:
#df_edgelist['inv_CH']=1/df_edgelist['CH']
new_df5.head()

,ID,lon,lat,Edge_ID,VD,sky,bldg,road,tree,Cl,NetCl,FP,CH,source_ID,target_ID,inv_CH
0,0,530462.00,200000.03,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.693147,0.0,1.0,1.442695
1,1,530498.99,200211.97,4.0,0.035506,29889.0,37920.0,3844.0,43543.0,0.0,1.0,0.000283,17.862410,1.0,5.0,0.055983
2,2,531525.98,200009.05,2.0,-0.087436,56167.0,23552.0,30815.0,15731.0,2.0,1.0,0.001773,17.562648,2.0,3.0,0.056939
3,3,531528.98,200000.00,65378.0,-0.037550,48892.0,25853.0,38137.0,22750.0,2.0,1.0,0.000493,17.562648,3.0,56957.0,0.056939
4,4,530191.96,200566.04,3.0,0.007294,30940.0,4670.0,24543.0,49084.0,0.0,1.0,0.000011,18.219468,4.0,1.0,0.054886


In [320]:
new_df5=pd.merge(df_nodelist,df_edgelist,how='left',left_on='ID',right_on='source_ID')
new_df6=new_df5[['ID','inv_CH','CH','road','Cl','bldg','tree']]
new_df6=new_df6.rename(columns={'inv_CH':'src_inv_CH','CH':'src_CH','road':'src_road','Cl':'src_Cl','bldg':'src_bldg','tree':'src_tree'})
new_df6=pd.merge(new_df6,df_edgelist,how='left',left_on='ID',right_on='target_ID')
new_df6=new_df6[['ID','src_inv_CH','src_CH','src_road','src_Cl','src_bldg','src_tree','inv_CH','CH','road','Cl','bldg','tree']]
new_df6=new_df6.rename(columns={'inv_CH':'tgt_inv_CH','CH':'tgt_CH','road':'tgt_road','Cl':'tgt_Cl','bldg':'tgt_bldg','tree':'tgt_tree'})

In [321]:
new_df6.head()

,ID,src_inv_CH,src_CH,src_road,src_Cl,src_bldg,src_tree,tgt_inv_CH,tgt_CH,tgt_road,tgt_Cl,tgt_bldg,tgt_tree
0,0,1.442695,0.693147,0.0,0.0,0.0,0.0,0.053879,18.559957,10211.0,0.0,21699.0,39833.0
1,1,0.055983,17.862410,3844.0,0.0,37920.0,43543.0,1.442695,0.693147,0.0,0.0,0.0,0.0
2,1,0.055983,17.862410,3844.0,0.0,37920.0,43543.0,0.054886,18.219468,24543.0,0.0,4670.0,49084.0
3,2,0.056939,17.562648,30815.0,2.0,23552.0,15731.0,0.056200,17.793498,39686.0,4.0,15820.0,8680.0
4,3,0.056939,17.562648,38137.0,2.0,25853.0,22750.0,0.056939,17.562648,30815.0,2.0,23552.0,15731.0


In [322]:
new_df6=new_df6.fillna(0)
# we are taking the max here
new_df7=new_df6.groupby('ID').max()
#new_df7=new_df6.groupby('ID').mean()


In [323]:
new_df7['inv_CH']=1/(np.abs(new_df7['src_inv_CH']+new_df7['tgt_inv_CH']+0.01))**2
new_df7['CH']=1/(np.abs(new_df7['src_CH']+new_df7['tgt_CH']+0.01))**2
new_df7['bldg']=1/(np.abs(new_df7['src_bldg']-new_df7['tgt_bldg']+0.01))**2
new_df7['tree']=1/(np.abs(new_df7['src_tree']-new_df7['tgt_tree']+0.01))**2
new_df7['road']=1/(np.abs(new_df7['src_road']-new_df7['tgt_road']+0.01))**2
new_df7['Cl']=1/(np.abs(new_df7['src_Cl']-new_df7['tgt_Cl']+0.01))**2

#new_df7['inv_CH']=new_df7[["src_inv_CH","tgt_inv_CH"]].max(axis=1)
#new_df7['CH']=new_df7[["src_CH","tgt_CH"]].max(axis=1)
#new_df7


In [324]:
del new_df7['src_inv_CH']
del new_df7['tgt_inv_CH']
del new_df7['src_CH']
del new_df7['tgt_CH']
del new_df7['src_road']
del new_df7['tgt_road']
del new_df7['src_Cl']
del new_df7['tgt_Cl']
del new_df7['src_bldg']
del new_df7['tgt_bldg']
del new_df7['src_tree']
del new_df7['tgt_tree']

In [325]:
new_df8=new_df4[['source_Edge_ID','target_Edge_ID','freq_ID']]

In [386]:
new_df7

,inv_CH,CH,bldg,tree,road,Cl
ID,,,,,,
0,0.440574,0.002695,2.123836e-09,6.302519e-10,9.591009e-09,10000.000000
1,0.439346,0.000768,9.045164e-10,3.257056e-08,2.334005e-09,10000.000000
2,65.948732,0.000800,1.672689e-08,2.011395e-08,1.270737e-08,0.252519
3,65.164539,0.000810,1.888700e-07,2.029777e-08,1.865260e-08,10000.000000
4,237.516270,0.003009,4.585264e-08,4.150686e-10,1.660139e-09,10000.000000
5,49.902394,0.001034,2.218500e-09,3.911137e-03,2.126771e-09,0.980296
6,49.749879,0.001038,1.324251e-06,8.235111e-10,4.988794e-09,0.110374
7,38.033429,0.001322,2.606009e-09,6.101618e-09,2.574105e-09,10000.000000
8,44.291940,0.001229,1.070743e-08,3.113831e-08,2.567593e-09,10000.000000


In [388]:
new_df8.to_csv('GLA/final/dual_graph_freqID.csv',index=False)
new_df7.to_csv('GLA/final/nodelist_process.csv',index=False)

In [4]:
new_df7=pd.read_csv('GLA/final/nodelist_process.csv')
new_df8=pd.read_csv('dual_graph_freqID.csv')

In [5]:
# now get the choice value of the junction node as the choice value 
new_df8=pd.merge(new_df8,new_df7,how='left',left_on='freq_ID',right_index=True)

In [6]:
# 42606 is approximately 20%
len(new_df8.loc[new_df8['road'] < 0.0000000036])

51161

In [7]:
# 42606 is approximately 20%
len(new_df8.loc[new_df8['tree'] < 0.000000005])

50800

In [8]:
# 42606 is approximately 20%
len(new_df8.loc[new_df8['bldg'] < 0.000000002])

51412

In [9]:
len(new_df8.loc[new_df8['CH'] < 0.00085])

50724

In [10]:
len(new_df8.loc[new_df8['Cl'] < 2])

48740

In [12]:
new_df8.max()

Unnamed: 0        213031.000000
source_Edge_ID    114018.000000
target_Edge_ID    114018.000000
freq_ID            84851.000000
inv_CH               334.266756
CH                     2.022588
bldg               10000.000000
tree               10000.000000
road               10000.000000
Cl                 10000.000000
dtype: float64

In [357]:
new_df8['new_CH']=np.NaN
new_df8.loc[new_df8['CH'] < 0.00085, 'new_CH'] = 1
new_df8.loc[new_df8['CH'] >= 0.00085, 'new_CH'] = 10

In [358]:
new_df8['new_road']=np.NaN
new_df8.loc[new_df8['road'] < 0.0000000036, 'new_road'] = 1
new_df8.loc[new_df8['road'] >= 0.0000000036, 'new_road'] = 10

In [359]:
new_df8['new_Cl']=np.NaN
new_df8.loc[new_df8['Cl'] < 2, 'new_Cl'] = 1
new_df8.loc[new_df8['Cl'] >= 2, 'new_Cl'] = 10

In [362]:
new_df8['new_tree']=np.NaN
new_df8.loc[new_df8['tree'] < 0.000000005, 'new_tree'] = 1
new_df8.loc[new_df8['tree'] >= 0.000000005, 'new_tree'] = 10

In [363]:
new_df8['new_bldg']=np.NaN
new_df8.loc[new_df8['bldg'] < 0.000000002, 'new_bldg'] = 1
new_df8.loc[new_df8['bldg'] >= 0.000000002, 'new_bldg'] = 10

In [364]:
#new_df7.to_csv('GLA/Junction_nodes_CH.csv')

In [365]:
# now calculate modularity optimisation based on the junction nodes
DG=nx.from_pandas_dataframe(new_df8, 'source_Edge_ID', 'target_Edge_ID')
partition = community.best_partition(DG)
DG=nx.from_pandas_dataframe(new_df8, 'source_Edge_ID', 'target_Edge_ID', 'new_road')
partition_new_road = community.best_partition(DG,weight='new_road')
DG=nx.from_pandas_dataframe(new_df8, 'source_Edge_ID', 'target_Edge_ID', 'new_CH')
partition_new_CH = community.best_partition(DG,weight='new_CH')
DG=nx.from_pandas_dataframe(new_df8, 'source_Edge_ID', 'target_Edge_ID', 'new_Cl')
partition_new_Cl = community.best_partition(DG,weight='new_Cl')
DG=nx.from_pandas_dataframe(new_df8, 'source_Edge_ID', 'target_Edge_ID', 'new_bldg')
partition_new_bldg = community.best_partition(DG,weight='new_bldg')
DG=nx.from_pandas_dataframe(new_df8, 'source_Edge_ID', 'target_Edge_ID', 'new_tree')
partition_new_tree = community.best_partition(DG,weight='new_tree')


df_partition = pd.DataFrame(list(partition.items()), columns=['NodeID', 'Com_Topo'])
df_partition_new_road = pd.DataFrame(list(partition_new_road.items()), columns=['NodeID', 'Com_new_road'])
df_partition_new_CH = pd.DataFrame(list(partition_new_CH.items()), columns=['NodeID', 'Com_new_CH'])
df_partition_new_Cl = pd.DataFrame(list(partition_new_Cl.items()), columns=['NodeID', 'Com_new_Cl'])
df_partition_new_bldg = pd.DataFrame(list(partition_new_bldg.items()), columns=['NodeID', 'Com_new_bldg'])
df_partition_new_tree = pd.DataFrame(list(partition_new_tree.items()), columns=['NodeID', 'Com_new_tree'])

df_com = pd.merge(df_partition,df_partition_new_road)
df_com = pd.merge(df_com,df_partition_new_CH)
df_com = pd.merge(df_com,df_partition_new_Cl)
df_com = pd.merge(df_com,df_partition_new_bldg)
df_com = pd.merge(df_com,df_partition_new_tree)

df_com.to_csv('GLA/GLA_dual_communities_avg_nodes_CH31.csv')

In [84]:
new_df8.head()

,source_Edge_ID,target_Edge_ID,freq_ID,inv_CH,CH
0,3.0,1.0,1,0.439346,0.000768
1,3.0,4.0,1,0.439346,0.000768
2,14.0,4451.0,22,40.365379,0.001355
3,4453.0,16.0,26,0.119285,0.512916
4,20.0,4457.0,34,0.119285,0.512916


In [85]:
new_df9.head()

,index,source_Edge_ID,target_Edge_ID,freq_ID,inv_CH,CH
0,89099,93294.0,13876.0,12920,27.885799,0.000941
1,11298,51720.0,51719.0,45748,48.463956,0.001116
2,158306,110983.0,110998.0,84223,64.795136,0.000749
3,94508,56248.0,56244.0,48673,20.081552,0.000827
4,79216,46777.0,44255.0,39051,49.088363,0.000943


In [86]:
new_df8['CH']=new_df8['CH']*100
new_df9=new_df8.sample(frac=1)
new_df9=new_df9.reset_index()

In [87]:
new_df9

,index,source_Edge_ID,target_Edge_ID,freq_ID,inv_CH,CH
0,16036,47169.0,47127.0,41037,44.288432,0.096574
1,110696,101284.0,101285.0,81430,52.444463,0.102472
2,127828,10508.0,10507.0,10084,41.434867,0.103889
3,78919,15329.0,13823.0,12877,41.131829,0.111410
4,118008,64805.0,55496.0,48361,42.544396,0.071564
5,134942,67324.0,104971.0,58983,61.494268,0.062953
6,107157,50982.0,49368.0,42358,0.428184,0.135419
7,27576,36439.0,37657.0,32046,43.675255,0.098970
8,168451,41496.0,39429.0,34664,43.575388,0.100489
9,195795,25574.0,27320.0,23821,122.195784,0.570347


In [88]:
# now calculate modularity optimisation based on the junction nodes
DG=nx.from_pandas_dataframe(new_df9, 'source_Edge_ID', 'target_Edge_ID')
partition = community.best_partition(DG)
DG=nx.from_pandas_dataframe(new_df9, 'source_Edge_ID', 'target_Edge_ID', 'CH')
partition_CH = community.best_partition(DG,weight='CH')
DG=nx.from_pandas_dataframe(new_df9, 'source_Edge_ID', 'target_Edge_ID', 'inv_CH')
partition_inv_CH = community.best_partition(DG,weight='inv_CH')
df_partition = pd.DataFrame(list(partition.items()), columns=['NodeID', 'Com_Topo'])
df_partition_CH = pd.DataFrame(list(partition_CH.items()), columns=['NodeID', 'Com_CH'])
df_partition_inv_CH = pd.DataFrame(list(partition_inv_CH.items()), columns=['NodeID', 'Com_invCH'])
df_com = pd.merge(df_partition,df_partition_CH)
df_com = pd.merge(df_com,df_partition_inv_CH)
df_com.to_csv('GLA/GLA_dual_communities_avg_nodes_CH13.csv')

## 12. Dual Graph communities using junction node choice (DX) - not needed
* take node weights of choice as dual graph weights to calculate modularity